## 目标:

利用机器学习，提高GNSS定位准确性，就是说修正卫星定位的经纬度

## 分析

In [ ]:
import pandas as pd 
import os

data_path = '/kaggle/input/google-smartphone-decimeter-challenge' 
print(os.listdir(data_path))

In [ ]:
train_location = pd.read_csv(data_path+'/baseline_locations_train.csv')
test_location = pd.read_csv(data_path+'/baseline_locations_test.csv')

In [ ]:
train_location.head()

In [ ]:
# 检查nan值
train_location.isna().any()

In [ ]:
train_location.collectionName.nunique()

In [ ]:
train_location.groupby("collectionName")['collectionName'].count()

In [ ]:
import plotly.express as pe

# 计算每个collectionName的数量
parent_folder = train_location.groupby("collectionName")['collectionName'].count().reset_index(name = 'Count')
bar_chart = pe.bar(parent_folder, x='collectionName', y='Count', title = '计算每个collectionName的数量')
bar_chart.show()

In [ ]:
train_location.phoneName.unique()

In [ ]:
import plotly.express as pe

# 对phoneName进行分组运算，计算每一个品牌手机所占的比例
phones = train_location.groupby('phoneName')['phoneName'].count().reset_index(name='count of phone used')
pie_chart = pe.pie(phones, values='count of phone used', names='phoneName', title='使用的手机比例')
pie_chart.show()

### 地图

In [ ]:
import folium

m = folium.Map(location=[37.453128,-122.154313], tiles='openstreetmap', zoom_start = 10)

sample_locations_test = train_location.sample(200).reset_index(drop = True)
sample_locations_train = test_location.sample(200).reset_index(drop = True)

for j in range(len(sample_locations_test)):
    try:
        folium.Marker(location=[sample_locations_test['latDeg'][j],
                                sample_locations_test['lngDeg'][j]],
                        popup=sample_locations_test['collectionName'][j],
                        icon = folium.Icon(prefix = 'fa', icon = "map-pin", color = 'lightblue'),
                        fill_color='#132b5e', num_sides=3, radius=3).add_to(m)
    except:
        continue
        
m

In [ ]:
for j in range(len(sample_locations_train)):
    try:
        folium.Marker(location=[sample_locations_train['latDeg'][j],
                                sample_locations_train['lngDeg'][j]],
                        popup=sample_locations_train['collectionName'][j],
                        icon = folium.Icon(prefix = 'fa', icon = "map-pin", color = 'red'),
                        fill_color='#132b5e', num_sides=3, radius=3).add_to(m)
    except:
        continue

m

### 路径

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import plotly.express as px
import seaborn as sns

In [ ]:
dir_path = '../input/google-smartphone-decimeter-challenge'
file_train = os.path.join(dir_path, "baseline_locations_train.csv")
file_test = os.path.join(dir_path, "baseline_locations_test.csv")
file_sub = os.path.join(dir_path, "sample_submission.csv")

data_train = pd.read_csv(file_train)
data_test = pd.read_csv(file_test)
data_sub = pd.read_csv(file_sub)

path_data = Path(dir_path)
path_truth_data = (path_data/'train').rglob('ground_truth.csv')

In [ ]:
data_truth_list =[]
for file_name in tqdm(path_truth_data, total=73):
    data_file = pd.read_csv(file_name)
    data_truth_list.append(data_file)
    
data_train_truth = pd.concat(data_truth_list, ignore_index=True)

In [ ]:
train_columns = ['collectionName', 'phoneName', 'millisSinceGpsEpoch','latDeg','lngDeg']
merge_columns = ['collectionName', 'phoneName', 'millisSinceGpsEpoch']
pd_train = data_train_truth[train_columns].merge(data_train, on=merge_columns, suffixes=("_truth",""))

pd_test = data_test

In [ ]:
pd_train.head()

In [ ]:
def visualize_trafic(df, zoom=9):
    fig1 = px.scatter_mapbox(df,
                            lat="latDeg",
                            lon="lngDeg",
                            
                            color="phone",
                            labels="phone",
                            
                            zoom=zoom,
                            # center=center,
                            height=600,
                            width=800)
    fig2 = px.scatter_mapbox(df,
                            lat="latDeg_truth",
                            lon="lngDeg_truth",
                            
                            color="phone",
                            labels="phone",
                            
                            zoom=zoom,
                            # center=center,
                            height=600,
                            width=800)
    fig1.update_layout(mapbox_style='stamen-terrain')
    fig1.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig1.update_layout(title_text="GPS trafic")
    
    fig2.update_layout(mapbox_style='stamen-terrain')
    fig2.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig2.update_layout(title_text="GPS trafic")
    fig1.show()
    fig2.show()

In [ ]:
visualize_trafic(pd_train)

In [ ]:
# def create_gif_track_on_map(df, gdf_map, git_path):
#     """ Create git animation of phone track on bayarea map.
#     """

#     fig, ax = plt.subplots()
#     df["geometry"] = [Point(lngDeg, latDeg) for lngDeg, latDeg in zip(df["lngDeg"], df["latDeg"])]
#     gdf = GeoDataFrame(df)
#     gdf.plot(color="lightskyblue", ax=ax)
#     imgs = []  
#     gdf_map.plot(color='none', edgecolor='gray', zorder=3, ax=ax)
    
#     # Here, (x,y) coordinates are thinned out!!!
#     for i in range(0, len(gdf), 10):
#         # plot data on map
#         p = ax.plot(gdf.iloc[i]["lngDeg"], gdf.iloc[i]["latDeg"], 
#                     color = 'dodgerblue', marker = 'o', markersize = 8)
#         imgs.append(p)

#     # Create animation & save it
#     ani = animation.ArtistAnimation(fig, imgs, interval=200)
#     ani.save(git_path, writer='imagemagick', dpi = 300)

## Features

In [ ]:
pd_train = pd_train.sort_values(['collectionName', 'phoneName', 'millisSinceGpsEpoch'])
pd_test = pd_test.sort_values(['collectionName', 'phoneName', 'millisSinceGpsEpoch'])

#### 异常值处理

In [ ]:
def calc_haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

def add_distance_diff(df):
    df['latDeg_prev'] = df['latDeg'].shift(1)
    df['latDeg_next'] = df['latDeg'].shift(-1)
    df['lngDeg_prev'] = df['lngDeg'].shift(1)
    df['lngDeg_next'] = df['lngDeg'].shift(-1)
    df['phone_prev'] = df['phone'].shift(1)
    df['phone_next'] = df['phone'].shift(-1)
    
    df['dist_prev'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_prev'], df['lngDeg_prev'])
    df['dist_next'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_next'], df['lngDeg_next'])
    
    df.loc[df['phone']!=df['phone_prev'], ['latDeg_prev', 'lngDeg_prev', 'dist_prev']] = np.nan
    df.loc[df['phone']!=df['phone_next'], ['latDeg_next', 'lngDeg_next', 'dist_next']] = np.nan
    
    return df

In [ ]:
th = 40
pd_train = add_distance_diff(pd_train)
pd_train.loc[((pd_train['dist_prev'] > th) \
             & (pd_train['dist_next'] > th)), ['latDeg', 'lngDeg']] = np.nan

pd_test = add_distance_diff(pd_test)
pd_test.loc[((pd_test['dist_prev'] > th)\
             & (pd_test['dist_next'] > th)), ['latDeg', 'lngDeg']] = np.nan

KalmanFilter Doc： https://simdkalman.readthedocs.io/en/latest/

In [ ]:
import simdkalman

T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)

def apply_kf_smoothing(df, name, kf_=kf):
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().values
    for collection, phone in unique_paths:
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].values
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        df.loc[cond, 'latDeg_'+name] = smoothed.states.mean[0, :, 0]
        df.loc[cond, 'lngDeg_'+name] = smoothed.states.mean[0, :, 1]
    return df

In [ ]:
pd_train = apply_kf_smoothing(pd_train, "_kf", kf)
pd_test = apply_kf_smoothing(pd_test, "_kf", kf)

#### 类别

In [ ]:

dict_ = dict(zip(set(pd_test['collectionName']), range(len(set(pd_test['collectionName'])))))
pd_train['collectionName_cat01'] = pd_train['collectionName'].map(dict_)
pd_test['collectionName_cat01'] = pd_test['collectionName'].map(dict_)

In [ ]:
dict_ = dict(zip(set(pd_test['phoneName']), range(len(set(pd_test['phoneName'])))))
pd_train['phoneName_cat01'] = pd_train['phoneName'].map(dict_)
pd_test['phoneName_cat01'] = pd_test['phoneName'].map(dict_)

#### shift

In [ ]:
shift_list = [-6,-5,-4,-3,-2,-1,1,2,3,4,5,6]
shift_columns = ['millisSinceGpsEpoch','latDeg','lngDeg','heightAboveWgs84EllipsoidM']
for shift_i in shift_list:
    tmp_test = pd_test.groupby(['collectionName','phoneName']).shift(shift_i)
    for col in shift_columns:
        pd_test[col+'_shift_'+str(shift_i)] = tmp_test[col]
        
    tmp_train = pd_train.groupby(['collectionName','phoneName']).shift(shift_i)
    for col in shift_columns:
        pd_train[col+'_shift_'+str(shift_i)] = tmp_train[col]

In [ ]:
pd_train.head(3)

#### groupby

In [ ]:
groupby_columns = ['millisSinceGpsEpoch']
values_columns = ['latDeg','lngDeg','heightAboveWgs84EllipsoidM']
function_names = ['sum','mean','size','min','max']
data_list = [pd_train, pd_test]
data_out = []
for data in data_list:
    tmp = data.\
        groupby(groupby_columns)[values_columns].\
        agg(function_names).\
        reset_index()

    tmp_columns = groupby_columns.copy()
    for i in values_columns:
        for j in function_names:
            tmp_columns.append('_'.join(groupby_columns) + '_' + i + '_' + j )

    tmp.columns = tmp_columns
    
    data = data.merge(tmp, on=groupby_columns)
    data_out.append(data)
pd_train, pd_test = data_out

In [ ]:
groupby_columns = ['collectionName','phoneName']
values_columns = ['latDeg','lngDeg','heightAboveWgs84EllipsoidM']
function_names = ['sum','mean','size','min','max']
data_list = [pd_train, pd_test]
data_out = []
for data in data_list:
    tmp = data.\
    groupby(groupby_columns)[values_columns].\
    agg(function_names).\
    reset_index()

    tmp_columns = groupby_columns.copy()
    for i in values_columns:
        for j in function_names:
            tmp_columns.append('_'.join(groupby_columns) + '_' + i + '_' + j )

    tmp.columns = tmp_columns
    
    data = data.merge(tmp,on=groupby_columns)
    data_out.append(data)
pd_train, pd_test = data_out

In [ ]:
groupby_columns = ['collectionName']
values_columns = ['latDeg','lngDeg','heightAboveWgs84EllipsoidM']
function_names = ['sum','mean','size','min','max']
data_list = [pd_train, pd_test]
data_out = []
for data in data_list:
    tmp = data.\
    groupby(groupby_columns)[values_columns].\
    agg(function_names).\
    reset_index()

    tmp_columns = groupby_columns.copy()
    for i in values_columns:
        for j in function_names:
            tmp_columns.append('_'.join(groupby_columns) + '_' + i + '_' + j )

    tmp.columns = tmp_columns
    
    data = data.merge(tmp,on=groupby_columns)
    data_out.append(data)
pd_train, pd_test = data_out

In [ ]:
groupby_columns = ['phoneName']
values_columns = ['latDeg','lngDeg','heightAboveWgs84EllipsoidM']
function_names = ['sum','mean','size','min','max']
data_list = [pd_train, pd_test]
data_out = []
for data in data_list:
    tmp = data.\
    groupby(groupby_columns)[values_columns].\
    agg(function_names).\
    reset_index()

    tmp_columns = groupby_columns.copy()
    for i in values_columns:
        for j in function_names:
            tmp_columns.append('_'.join(groupby_columns) + '_' + i + '_' + j )

    tmp.columns = tmp_columns
    
    data = data.merge(tmp,on=groupby_columns)
    data_out.append(data)
pd_train, pd_test = data_out

#### 滑动平均

In [ ]:
groupby_columns = ['collectionName','phoneName']
rolling_columns = ['latDeg','lngDeg','heightAboveWgs84EllipsoidM']
window_list = [2,3,5,7,10]
for i in window_list:
    for data in [pd_train, pd_test]:
        temp = data.groupby(groupby_columns)[rolling_columns].rolling(i, center=True).mean().reset_index()
        for col in rolling_columns:
            data[col+'_rolling_'+str(i)] = temp[col]

#### split

In [ ]:
remove_columns = ['collectionName','phoneName','latDeg_truth','lngDeg_truth','phone']
features_columns = [col for col in pd_test.columns if col not in remove_columns]

X_train = pd_train[features_columns]
X_test = pd_test[features_columns]
y_latDeg = pd_train['latDeg_truth']
y_lngDeg = pd_train['lngDeg_truth']

xtr1,xval1,ytr1,yval1 = train_test_split(X_train, y_latDeg, test_size=0.3, random_state=10)
xtr2,xval2,ytr2,yval2 = train_test_split(X_train, y_lngDeg, test_size=0.3, random_state=10)

## Model

In [ ]:
params = {
    'objective': 'mae',
    'max_bin': 600,
    'learning_rate': 0.02,
    'num_leaves': 80
}

lgb_train = lgb.Dataset(xtr1, ytr1)
lgb_eval = lgb.Dataset(xval1, yval1, reference=lgb_train)

model_latDeg = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=25,
    num_boost_round=10000,
    early_stopping_rounds=10
)

## 

In [ ]:
params = {
    'objective': 'mae',
    'max_bin': 600,
    'learning_rate': 0.02,
    'num_leaves': 80
}

lgb_train = lgb.Dataset(xtr2, ytr2)
lgb_eval = lgb.Dataset(xval2, yval2, reference=lgb_train)

model_lngDeg = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=25,
    num_boost_round=10000,
    early_stopping_rounds=10
)

In [ ]:
y_pred = model_latDeg.predict(xval1)
y_true = np.array(yval1)
mean_absolute_error(y_true, y_pred)

In [ ]:
y_pred = model_lngDeg.predict(xval2)
y_true = np.array(yval2)
mean_absolute_error(y_true, y_pred)

In [ ]:
y_lngDeg = model_lngDeg.predict(X_test)
y_latDeg = model_latDeg.predict(X_test)

submission = pd_test[['phone','millisSinceGpsEpoch']]
submission['latDeg'] = list(y_latDeg)
submission['lngDeg'] = list(y_lngDeg)


df_sub = data_sub[['phone', 'millisSinceGpsEpoch']].merge(submission,
                                                          on=['phone', 'millisSinceGpsEpoch'],
                                                          how='inner')
df_sub.to_csv("./submission.csv", index=False)

深度模型：https://www.kaggle.com/jeongyoonlee/google-smartphone-decimeter-eda-keras-tpu